In [2]:
import moxing as mox
data_url = "s3://nlp-pmq/nlp/final/data"
mox.file.copy_parallel(src_url = data_url,dst_url='./data')

INFO:root:Using MoXing-v2.0.0.rc2.4b57a67b-4b57a67b
INFO:root:Using OBS-Python-SDK-3.20.9.1
INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:Listing OBS: 8000
INFO:root:Listing OBS: 9000
INFO:root:Listing OBS: 10000
INFO:root:Listing OBS: 11000
INFO:root:Listing OBS: 12000
INFO:root:Listing OBS: 13000
INFO:root:Listing OBS: 14000
INFO:root:Listing OBS: 15000
INFO:root:Listing OBS: 16000
INFO:root:Listing OBS: 17000
INFO:root:Listing OBS: 18000
INFO:root:Listing OBS: 19000
INFO:root:Listing OBS: 20000
INFO:root:Listing OBS: 21000
INFO:root:Listing OBS: 22000
INFO:root:Listing OBS: 23000
INFO:root:Listing OBS: 24000
INFO:root:Listing OBS: 25000
INFO:root:Listing OBS: 26000
INFO:root:Listing OBS: 27000
INFO:root:Listing OBS: 28000
INFO:root:Listing OBS: 29000
INFO:root:Listing OBS: 30000
INFO:root:Listing OBS: 31000
INFO:root:Listing 

In [3]:
code_url = "s3://nlp-pmq/nlp/final/src"
mox.file.copy_parallel(src_url = code_url,dst_url='./src')

###### 

In [4]:
import numpy as np
config = {
    # base info about dataset
    'train_num':44375,
    'test_num':21435,
    'test_num':21435,

    # paradim setting
    'sentence_len':8,
    'use_stopwords':True,

}
stoplist = set('a the this that these those of on in to for'.split())

In [5]:
import src.utils as utils
import pandas as pd
import re
from copy import deepcopy

Q_set,A_set = utils.load_json_file(path = './data',cleaned=False)
for item in Q_set:
    qst = item['question']
    qst = qst.lower()
    qst = re.sub(r"[^a-zA-Z]+", r" ",qst)
    qst = qst.strip()
    qst_vocab = qst.split(' ')
    qst_vocab = utils.sentence_align(qst_vocab,config['sentence_len'])
    item['question'] = deepcopy(qst_vocab)
    
print(len(Q_set))

Q_df = pd.DataFrame(Q_set)
A_df = pd.DataFrame(A_set)

Q_A_df = Q_df.merge(A_df,on = ['question_id','image_id'])
Q_A_df = Q_A_df[['question_id','image_id','question','multiple_choice_answer']]
Q_A_df['annotation'] = Q_A_df['multiple_choice_answer']
Q_A_df = Q_A_df[['question_id','image_id','question','annotation']]

print(Q_A_df.shape)
Q_A_df.head()

train : 44375, eval : 21435, test : 21435
87245
(87245, 4)


,question_id,image_id,question,annotation
0,393223003,393223,"[is, she, brushing, <INS>, <INS>, <INS>, <INS>...",yes
1,393227001,393227,"[what, is, this, man, riding, on, <INS>, <INS>]",skateboard
2,131074002,131074,"[are, the, walls, done, in, a, summery, color]",yes
3,393230003,393230,"[what, is, the, man, doing, <INS>, <INS>, <INS>]",skiing
4,393230005,393230,"[what, is, the, person, wearing, <INS>, <INS>,...",skis


In [6]:
!pip install gensim

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [7]:
import src.word2vec as word2vec
word2vec_model = word2vec.load_word2vec('./src/word_vec/embedding.txt')
def qst2tenser(qst,model):
    '''
    note: 尚未测试,可能有bug
    '''
    qst_tensor = []
    for item in qst:
        if item == '<INS>':
            qst_tensor.append([[0]*100])
        else :
            qst_tensor.append(model[item])
    return qst_tensor

rst = []
for item in  Q_A_df['question']:
    rst.append(qst2tenser(item,word2vec_model))

Q_A_df['question_tensor'] = rst
Q_A_df.head()
# word2vec_model

/home/ma-user/miniconda3/envs/MindSpore-python3.7-aarch64/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
INFO:gensim.models.keyedvectors:loading projection weights from ./src/word_vec/embedding.txt
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (10408, 100) matrix of type float32 from ./src/word_vec/embedding.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-06-25T15:14:48.988139', 'gensim': '4.2.0', 'python': '3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:15:32) \n[GCC 7.5.0]', 'platform': 'Linux-4.19.36-vhulk1907.1.0.h619.eulerosv2r8.aarch64-aarch64-with-centos-2.0-SP8', 'event': 'load_word2vec_format'}


,question_id,image_id,question,annotation,question_tensor
0,393223003,393223,"[is, she, brushing, <INS>, <INS>, <INS>, <INS>...",yes,"[[-0.62656933, 0.6822378, 0.31856343, 0.168713..."
1,393227001,393227,"[what, is, this, man, riding, on, <INS>, <INS>]",skateboard,"[[-0.17337483, 0.65863776, -0.0061338176, 0.31..."
2,131074002,131074,"[are, the, walls, done, in, a, summery, color]",yes,"[[-0.6145737, 0.4990958, -0.13994391, 0.048096..."
3,393230003,393230,"[what, is, the, man, doing, <INS>, <INS>, <INS>]",skiing,"[[-0.17337483, 0.65863776, -0.0061338176, 0.31..."
4,393230005,393230,"[what, is, the, person, wearing, <INS>, <INS>,...",skis,"[[-0.17337483, 0.65863776, -0.0061338176, 0.31..."


In [8]:
from src.data_process import create_dataset
from src.image_set import ImageSet
import os
print (os.getcwd())

/home/ma-user/work


In [ ]:
images = ImageSet(256)
dataset = create_dataset(batch_size = 32, images = images, Q_A_df = Q_A_df, mode = "train", drop_remainder = True)

In [ ]:
print(dataset)